In [1]:
#Google Colab Library
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Kaggle"

In [4]:
%cd /content/drive/My Drive/Kaggle

/content/drive/My Drive/Kaggle


In [5]:
!kaggle datasets download -d amananandrai/ag-news-classification-dataset

ag-news-classification-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Kaggle/AG News.csv',encoding = "ISO-8859-1")

In [7]:
df.head()

,Class Index,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [8]:
df = df.drop(columns='Title')
df = df.drop_duplicates(subset = ["Description"])



In [9]:
category = pd.get_dummies(df['Class Index'])
df = pd.concat([df , category], axis=1)
df = df.drop(columns='Class Index')
df

,Description,1,2,3,4
0,Unions representing workers at Turner Newall...,0,0,1,0
1,"SPACE.com - TORONTO, Canada -- A second\team o...",0,0,0,1
2,AP - A company founded by a chemistry research...,0,0,0,1
3,AP - It's barely dawn when Mike Fitzpatrick st...,0,0,0,1
4,AP - Southern California's smog-fighting agenc...,0,0,0,1
...,...,...,...,...,...
7595,Ukrainian presidential candidate Viktor Yushch...,1,0,0,0
7596,With the supply of attractive pitching options...,0,1,0,0
7597,Like Roger Clemens did almost exactly eight ye...,0,1,0,0
7598,SINGAPORE : Doctors in the United States have ...,0,0,1,0


In [10]:
#Lower Casing
df['Description'] = df['Description'].str.lower()

In [11]:
#Punctuation Removal
df['Description'] = df['Description'].str.replace('[^\w\s]','')

In [12]:
desc = df['Description'].values
label = df[[1, 2,3, 4]].values

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(desc, label, test_size=0.2)

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(x_train) 


x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

x_train = pad_sequences(x_train) 
x_test = pad_sequences(x_test)

In [15]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.9 and logs.get('val_accuracy') > 0.9):
      print("\nAkurasi telah mencapai >90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [17]:
import numpy as np
num_epochs = 30
x_train = np.array(x_train)

hist = model.fit(x_train, y_train, epochs=num_epochs, 
                    validation_data=(x_test, y_test), callbacks=[callbacks], verbose=1)

Epoch 1/30
190/190 [==============================] - 10s 55ms/step - loss: 1.2887 - accuracy: 0.3323 - val_loss: 0.9579 - val_accuracy: 0.4806
Epoch 2/30
190/190 [==============================] - 11s 55ms/step - loss: 0.8690 - accuracy: 0.5923 - val_loss: 0.7600 - val_accuracy: 0.6504
Epoch 3/30
190/190 [==============================] - 11s 57ms/step - loss: 0.6533 - accuracy: 0.7103 - val_loss: 0.6207 - val_accuracy: 0.7683
Epoch 4/30
190/190 [==============================] - 11s 57ms/step - loss: 0.4573 - accuracy: 0.8380 - val_loss: 0.6103 - val_accuracy: 0.7788
Epoch 5/30
190/190 [==============================] - 11s 57ms/step - loss: 0.3012 - accuracy: 0.9045 - val_loss: 0.6683 - val_accuracy: 0.7808
Epoch 6/30
190/190 [==============================] - 11s 57ms/step - loss: 0.2242 - accuracy: 0.9327 - val_loss: 0.7629 - val_accuracy: 0.7702
Epoch 7/30
190/190 [==============================] - 12s 62ms/step - loss: 0.1962 - accuracy: 0.9440 - val_loss: 1.0591 - val_accuracy: